In [2]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
# Define the language model to use.
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [122]:
# Define the tools to use.
import json
import datetime
from collections import defaultdict
from langchain.agents import tool
from typing import Dict, Text, Any, List
from pydantic.v1 import BaseModel, Field
from langchain.tools.render import format_tool_to_openai_function



@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def get_peloton_classes() -> List[Dict[Text, Any]]:
    """Get recent Peloton classes."""
    response = json.load(open("peloton_classes.json", "r"))

    today = datetime.datetime.today().date()
    recent_classes = []
    # Create the instructors lookup.
    instructors = {i['id']: i['name'] for i in response['instructors']}
    for w in response['data']:
        workout_date = datetime.datetime.fromtimestamp(w['original_air_time']).date()

        if (today - workout_date).days > 7:
            break

        recent_classes.append(
            {
                'id': w['id'],
                'description': w['description'],
                'difficulty': w['difficulty_estimate'],
                'duration': w['duration'],
                'instructor': instructors[w['instructor_id']],
                'title': w['title'],
                'disciplie': w['fitness_discipline_display_name']
            }
        )
    return recent_classes


@tool(return_direct=False)
def get_recent_user_workouts() -> Dict[Text, Any]:
    """Get the user's Peloton workouts from the past week."""
    response = json.load(open("user_workouts.json", "r"))

    today = datetime.datetime.today().date()
    recent_workouts = defaultdict(list)
    for w in response['data']:
        workout_date = datetime.datetime.fromtimestamp(w['created_at']).date()

        # Only get workouts from the last 7 days.
        if (today - workout_date).days > 7:
            break

        if 'ride' in w:
            title = w['ride']['title']
        elif 'peloton' in w:
            title = w['peloton']['ride']['title']
        else:
            title = "Unknown"
            
        lbl = f"{workout_date}: {title}"

        recent_workouts[str(workout_date)].append(lbl)

    return json.dumps(recent_workouts)


class StackInput(BaseModel):
    recommended_workout: List[Dict[Text, Any]] = Field(description="the list of recommended classes.")


@tool(args_schema=StackInput)
def add_class_to_stack(recommended_workout: List[Dict[Text, Any]]):
    """Allows a user to add selected workout to the Peloton stack if the user explicitly asks to."""
    print(recommended_workout)


class RecommendInput(BaseModel):
    user_input: str = Field(description="the user message")


@tool
def recommend_workout(user_input: str):
    """Recommend a class for the user."""
    print(user_input)
    return "This is your recommended class."


tools = [get_word_length, get_recent_user_workouts, get_peloton_classes, add_class_to_stack, recommend_workout]



In [123]:
from typing import List, Optional

from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field


class Response(BaseModel):
    """Final response returned to the user."""

    answer: str = Field(description="The final answer to respond to the user")
    raw_data: Optional[Dict[Text, Any]] = Field(
        description="Raw JSON response from the tool if one is provided."
    )

# tools.append(convert_pydantic_to_openai_function(Response))
# tools

In [124]:
# Tell the LLM about the tools available via `bind`.
from langchain.tools.render import format_tool_to_openai_function

openai_tools = [format_tool_to_openai_function(t) for t in tools]
openai_tools.append(convert_pydantic_to_openai_function(Response))

llm_with_tools = llm.bind(functions=openai_tools)

In [125]:
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a personal trainer helping the user to achieve their fitness goals.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [130]:
import json

from langchain.schema.agent import AgentActionMessageLog, AgentFinish

def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    print(name)
    print(inputs)
    
    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [131]:
# Define the agent.
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.agents import AgentExecutor

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | parse
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [132]:
from langchain.agents import AgentExecutor

chat_history = []
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [133]:
input1 = "Recommend a new workout for me."
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

# print(result)



> Entering new AgentExecutor chain...
recommend_workout
{'user_input': "I'm looking for a new workout to try."}
I'm looking for a new workout to try.
This is your recommended class.I've found a recommended workout class for you to try. If you'd like more details about the class or want to add it to your Peloton stack, just let me know!

> Finished chain.


In [87]:
result.keys()

dict_keys(['input', 'chat_history', 'output', 'intermediate_steps'])

In [11]:
agent_executor.invoke({"input": "Yes", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `add_class_to_stack` with `{}`




ValidationError: 1 validation error for StackInput
recommended_workout
  field required (type=value_error.missing)